In [2]:
import pandas as pd
import requests

In [26]:
class HargaJatim:
    NAMA_BAHAN_POKOK = ['BERAS', 'Beras Bengawan', 'Beras Mentik', 'Beras IR 64', 'GULA PASIR',
                        'Gula Pasir Dalam Negri', 'MINYAK GORENG', 'Bimoli Botol / Kemasan (Sps)',
                        'Bimoli botol/Kemasan (sps)', 'Tanpa Merk / Minyak Curah', 'DAGING',
                        'Daging Sapi Murni', 'Daging Ayam Broiler', 'Daging Ayam Kampung',
                        'TELUR AYAM', 'Telur Ayam Ras / Petelur', 'Telur Ayam Kampung', 'SUSU',
                        'Susu Kental Manis', 'Susu Kental Manis Merk Bendera', 'Susu Kental Manis Merk Indomilk',
                        'Susu Bubuk', 'Susu Bubuk Merk Bendera (Instant)', 'Susu Bubuk Merk Indomilk (Instant)',
                        'JAGUNG PIPILAN KERING', 'GARAM BERYODIUM', 'Garam Bata', 'Garam Halus',
                        'TEPUNG TERIGU', 'Tepung Segitiga Biru (Kw Medium)', 'KACANG KEDELAI',
                        'Kacang Kedelai Eks Impor', 'Kacang Kedelai Lokal', 'MIE INSTANT',
                        'Indomie Rasa Kari Ayam', 'CABE', 'Cabe Keriting', 'Cabe Biasa',
                        'Cabe Rawit','BAWANG MERAH', 'BAWANG PUTIH', 'IKAN ASIN TERI', 'KACANG HIJAU', 'KACANG TANAH',
                        'KETELA POHON', 'SAYUR MAYUR', 'KOL/KUBIS', 'KENTANG', 'TOMAT', 'WORTEL',
                        'BUNCIS', 'SEMEN', 'Semen Gresik', 'Semen Tiga Roda', 'Semen Holcim',
                        'Semen Padang', 'Semen Tonasa', 'Semen Bosowa', 'IKAN SEGAR', 'Ikan Bandeng',
                        'Ikan Kembung', 'Ikan Tuna', 'ikan Tongkol', 'ikan cakalang', 'KAYU BALOK MERANTI (4 X 10)',
                        'PAPAN MERANTI (4m X 3cm X 20mm)', 'TRIPLEK (6MM)', 'BESI BETON (SNI MURNI)', 'Besi Beton 6 mm (12/9m)',
                        'Besi Beton 8 mm (12/9m)', 'Besi Beton 10 mm (12/9m)', 'Besi Beton 12 mm (12/9m)',
                        'PAKU', 'Paku Ukuran 10Cm', 'Paku Ukuran 2 Cm', 'Paku Ukuran 3Cm', 'Paku Ukuran 4Cm',
                        'Paku Ukuran 5Cm', 'Paku Ukuran 7Cm', 'GAS ELPIGI 3 Kg', 'PUPUK', 'Pupuk KCL Non Subsidi',
                        'Pupuk NPK Non Subsidi', 'Pupuk SP 35 Non Subsidi', 'Pupuk Urea Non Subsidi', 'Pupuk ZA Non Subsidi']
    
    def __init__(self):
        self._url = 'http://siskaperbapo.com/harga/tabel.nodesign/'
        self._dataframe = None
        
    def _post_request(self, tanggal):
        r = requests.post(self._url, 
                          data={'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
                                                                         'tanggal': tanggal,
                                                                         'kabkota': ''})
        return r.text
    
    def _cleaning_dataframe(self, request_text, tanggal):
        df = pd.read_html(request_text, header=0)[0]
        harga = df[['NAMA BAHAN POKOK', 'SATUAN', 'HARGA SEKARANG']]
        nama_bahan = pd.Series(HargaJatim.NAMA_BAHAN_POKOK)
        harga['NAMA BAHAN POKOK'] = nama_bahan
        harga = harga[~harga['SATUAN'].isnull()]
        harga['SATUAN'] =  harga['SATUAN'].str.title()
        harga['NAMA BAHAN POKOK'] =  harga['NAMA BAHAN POKOK'].str.title()
        harga['HARGA SEKARANG'] = harga['HARGA SEKARANG'].str.replace('.', '').astype('int32')
        harga['NAMA BAHAN POKOK'] = harga['NAMA BAHAN POKOK'] + ' ' + '(' + harga['SATUAN'] + ')'
        harga.index = range(len(harga))
        harga = harga[['NAMA BAHAN POKOK', 'HARGA SEKARANG']]
        harga_transpose = harga.transpose().copy()
        harga_transpose.columns = harga_transpose.iloc[0, :]
        harga_series = harga_transpose.iloc[1,:].copy()
        harga_series.name = tanggal
        harga_series.name = pd.to_datetime(harga_series.name)
        return harga_series
    
    def get_data(self, mulai, berakhir):
        list_tanggal = pd.date_range(mulai, berakhir)
        list_series = []
        for tanggal in list_tanggal:
            tanggal = tanggal.strftime('%Y-%m-%d')
            r = self._post_request(tanggal)
            series = self._cleaning_dataframe(r, tanggal)
            list_series.append(series)
        self._dataframe = pd.DataFrame(list_series)
        return self._dataframe
    
    def save_tocsv(self, name):
        self._dataframe.to_csv(name, index=False)
        


In [27]:
januari = HargaJatim()

In [28]:
januari.get_data('2017-01-01', '2017-01-31')

/home/rubiagatra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


NAMA BAHAN POKOK,Beras Bengawan (Kg),Beras Mentik (Kg),Beras Ir 64 (Kg),Gula Pasir Dalam Negri (Kg),Bimoli Botol / Kemasan (Sps) (620 Ml),Bimoli Botol/Kemasan (Sps) (2 Liter),Tanpa Merk / Minyak Curah (Kg),Daging Sapi Murni (Kg),Daging Ayam Broiler (Kg),Daging Ayam Kampung (Kg),...,Paku Ukuran 3Cm (Kg),Paku Ukuran 4Cm (Kg),Paku Ukuran 5Cm (Kg),Paku Ukuran 7Cm (Kg),Gas Elpigi 3 Kg (Kg),Pupuk Kcl Non Subsidi (Kg),Pupuk Npk Non Subsidi (Kg),Pupuk Sp 35 Non Subsidi (Kg),Pupuk Urea Non Subsidi (Kg),Pupuk Za Non Subsidi (Kg)
2017-01-01,10407,10299,8863,12637,11677,25445,11623,105444,29565,55500,...,14066,13578,13231,13038,17503,5753,6869,3716,3423,3108
2017-01-02,10409,10315,8874,12667,11631,25464,11644,105287,29514,55477,...,14133,13648,13351,13175,17503,5394,6846,3707,3454,3100
2017-01-03,10371,10281,8910,12609,11729,25471,11639,105372,29323,55885,...,13985,13577,13309,13064,17443,5581,6657,3488,3312,2943
2017-01-04,10421,10322,8921,12621,11711,25479,11644,105301,29288,55522,...,14042,13649,13413,13188,17438,5559,6647,3546,3336,2990
2017-01-05,10314,10244,8896,12610,11773,25433,11623,105486,29091,54845,...,14048,13594,13352,13125,17462,5516,6555,3395,3297,2863
2017-01-06,10305,10264,8924,12592,11878,25357,11627,105343,29366,54787,...,14111,13542,13272,13043,17471,5499,6521,3426,3313,2869
2017-01-07,10351,10281,8877,12616,11813,25464,11626,104675,29544,54422,...,14019,13477,13176,13027,17500,5501,6792,3460,3422,3001
2017-01-08,10343,10268,8895,12593,11769,25494,11614,105519,29481,54566,...,13995,13497,13179,12977,17485,5477,6781,3414,3386,2949
2017-01-09,10335,10261,8928,12657,11878,25436,11627,105532,29614,54917,...,14072,13599,13371,13164,17447,5544,6563,3373,3341,2870
2017-01-10,10344,10256,8956,12648,11886,25451,11662,105432,29757,54876,...,14052,13603,13402,13123,17462,5440,6583,4632,3303,2821


In [29]:
januari.save_tocsv('../data/Harga Bahan Pokok Jatim Januari 2017.csv')

In [30]:
harga_2017 = HargaJatim()

In [32]:
harga_2017.get_data('2017-01-01', '2017-12-31')

/home/rubiagatra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


NAMA BAHAN POKOK,Beras Bengawan (Kg),Beras Mentik (Kg),Beras Ir 64 (Kg),Gula Pasir Dalam Negri (Kg),Bimoli Botol / Kemasan (Sps) (620 Ml),Bimoli Botol/Kemasan (Sps) (2 Liter),Tanpa Merk / Minyak Curah (Kg),Daging Sapi Murni (Kg),Daging Ayam Broiler (Kg),Daging Ayam Kampung (Kg),...,Paku Ukuran 3Cm (Kg),Paku Ukuran 4Cm (Kg),Paku Ukuran 5Cm (Kg),Paku Ukuran 7Cm (Kg),Gas Elpigi 3 Kg (Kg),Pupuk Kcl Non Subsidi (Kg),Pupuk Npk Non Subsidi (Kg),Pupuk Sp 35 Non Subsidi (Kg),Pupuk Urea Non Subsidi (Kg),Pupuk Za Non Subsidi (Kg)
2017-01-01,10407,10299,8863,12637,11677,25445,11623,105444,29565,55500,...,14066,13578,13231,13038,17503,5753,6869,3716,3423,3108
2017-01-02,10409,10315,8874,12667,11631,25464,11644,105287,29514,55477,...,14133,13648,13351,13175,17503,5394,6846,3707,3454,3100
2017-01-03,10371,10281,8910,12609,11729,25471,11639,105372,29323,55885,...,13985,13577,13309,13064,17443,5581,6657,3488,3312,2943
2017-01-04,10421,10322,8921,12621,11711,25479,11644,105301,29288,55522,...,14042,13649,13413,13188,17438,5559,6647,3546,3336,2990
2017-01-05,10314,10244,8896,12610,11773,25433,11623,105486,29091,54845,...,14048,13594,13352,13125,17462,5516,6555,3395,3297,2863
2017-01-06,10305,10264,8924,12592,11878,25357,11627,105343,29366,54787,...,14111,13542,13272,13043,17471,5499,6521,3426,3313,2869
2017-01-07,10351,10281,8877,12616,11813,25464,11626,104675,29544,54422,...,14019,13477,13176,13027,17500,5501,6792,3460,3422,3001
2017-01-08,10343,10268,8895,12593,11769,25494,11614,105519,29481,54566,...,13995,13497,13179,12977,17485,5477,6781,3414,3386,2949
2017-01-09,10335,10261,8928,12657,11878,25436,11627,105532,29614,54917,...,14072,13599,13371,13164,17447,5544,6563,3373,3341,2870
2017-01-10,10344,10256,8956,12648,11886,25451,11662,105432,29757,54876,...,14052,13603,13402,13123,17462,5440,6583,4632,3303,2821


In [33]:
harga_2017.save_tocsv('../data/Harga Bahan Pokok Jatim selama 2017.csv')